In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import torchvision.utils as vutils

# --- Dataset Class ---
class PolymerDataset(Dataset):
    def __init__(self, input_dir, target_dir, size=(512, 512)):
        self.input_dir = input_dir
        self.target_dir = target_dir
        self.size = size
        self.filenames = sorted(os.listdir(input_dir))
        
        self.transform = transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor(),
        ])
    
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, idx):
        input_path = os.path.join(self.input_dir, self.filenames[idx])
        target_path = os.path.join(self.target_dir, self.filenames[idx])
        
        input_img = Image.open(input_path).convert('L')
        target_img = Image.open(target_path).convert('L')
        
        input_tensor = self.transform(input_img)
        target_tensor = self.transform(target_img)
        
        input_tensor = (input_tensor > 0.5).float() * 2 - 1
        target_tensor = (target_tensor > 0.5).float() * 2 - 1
        
        return input_tensor, target_tensor

In [3]:
# --- UNet Generator ---
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.conv_in = nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1, bias=False)
        
        # Downsampling
        self.down1 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128)
        )
        self.down2 = self._make_down_block(128, 256)
        self.down3 = self._make_down_block(256, 512)
        self.down4 = self._make_down_block(512, 512)
        self.down5 = self._make_down_block(512, 512)
        self.down6 = self._make_down_block(512, 512)

        # Middle
        self.middle = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512)
        )

        # Upsampling
        self.up1 = self._make_up_block(1024, 512, dropout=True)
        self.up2 = self._make_up_block(1024, 512, dropout=True)
        self.up3 = self._make_up_block(1024, 512, dropout=True)
        self.up4 = self._make_up_block(1024, 256)
        self.up5 = self._make_up_block(512, 128)
        self.up6 = self._make_up_block(256, 64)

        self.outermost = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 1, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def _make_down_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

    def _make_up_block(self, in_channels, out_channels, dropout=False):
        layers = [
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        ]
        if dropout:
            layers.append(nn.Dropout(0.5))
        return nn.Sequential(*layers)

    def forward(self, x):
        # Encoder
        x0 = self.conv_in(x)
        x1 = self.down1(x0)
        x2 = self.down2(x1)
        x3 = self.down3(x2)
        x4 = self.down4(x3)
        x5 = self.down5(x4)
        x6 = self.down6(x5)

        # Middle
        x = self.middle(x6)

        # Decoder with skip connections
        x = self.up1(torch.cat([x, x6], dim=1))
        x = self.up2(torch.cat([x, x5], dim=1))
        x = self.up3(torch.cat([x, x4], dim=1))
        x = self.up4(torch.cat([x, x3], dim=1))
        x = self.up5(torch.cat([x, x2], dim=1))
        x = self.up6(torch.cat([x, x1], dim=1))
        
        return self.outermost(torch.cat([x, x0], dim=1))

In [4]:
# --- PatchGAN Discriminator ---
class PatchGAN(nn.Module):
    def __init__(self):
        super(PatchGAN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(2, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(256, 512, kernel_size=4, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)
        )

    def forward(self, x):
        return self.net(x)

# --- Learning Rate Scheduler ---
class LambdaLR:
    def __init__(self, n_epochs, offset, decay_start_epoch):
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)

# --- Visualization Function ---
def visualize_results(inputs, targets, outputs, epoch, save_dir="training_results"):
    os.makedirs(save_dir, exist_ok=True)
    
    # Denormalize images
    inputs = (inputs.cpu().numpy() + 1) / 2
    targets = (targets.cpu().numpy() + 1) / 2
    outputs = (outputs.cpu().numpy() + 1) / 2
    
    plt.figure(figsize=(15, 5))
    for i in range(min(3, inputs.shape[0])):  # Show max 3 examples
        plt.subplot(3, 3, i*3+1)
        plt.imshow(inputs[i][0], cmap='gray')
        plt.title("Input")
        plt.axis('off')
        
        plt.subplot(3, 3, i*3+2)
        plt.imshow(targets[i][0], cmap='gray')
        plt.title("Target")
        plt.axis('off')
        
        plt.subplot(3, 3, i*3+3)
        plt.imshow(outputs[i][0], cmap='gray')
        plt.title("Output")
        plt.axis('off')
    
    plt.suptitle(f"Epoch {epoch+1}")
    plt.savefig(f"{save_dir}/epoch_{epoch+1}.png")
    plt.close()

In [5]:
# Load dataset
input_dir = '/kaggle/input/dataset1/sliced images/input'
target_dir = '/kaggle/input/dataset1/sliced images/target'
    

# Split dataset
dataset = PolymerDataset(input_dir, target_dir)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
gen = torch.Generator()
gen.manual_seed(30)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size], generator=gen)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [6]:
len(test_loader)

66

In [7]:
len(train_loader)

264

In [8]:
# --- Training Function with Visualization ---
def train():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Initialize models
    netG = UNet().to(device)
    netD = PatchGAN().to(device)
    
    # Loss functions
    criterion_GAN = nn.BCEWithLogitsLoss()
    criterion_L1 = nn.L1Loss()
    lambda_L1 = 20
    
    # Optimizers
    optimizer_G = torch.optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_D = torch.optim.Adam(netD.parameters(), lr=0.0001, betas=(0.5, 0.999))
    
    # Learning rate schedulers
    lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(
        optimizer_G, lr_lambda=LambdaLR(100, 0, 50).step)
    lr_scheduler_D = torch.optim.lr_scheduler.LambdaLR(
        optimizer_D, lr_lambda=LambdaLR(100, 0, 50).step)

    
    # Create fixed batch for visualization
    fixed_inputs, fixed_targets = next(iter(test_loader))
    fixed_inputs = fixed_inputs.to(device)
    
    # History tracking
    history = {
        'epoch': [],
        'G_loss': [],
        'D_loss': [],
        'D_real': [],
        'D_fake': [],
        'D_x': [],       # D(x) - average output on real images
        'D_G_z': [],     # D(G(z)) - average output on fake images
        'val_loss': []
    }
    
    # Training loop
    for epoch in range(100):
        netG.train()
        netD.train()
        
        # Initialize accumulators
        running_loss_G = 0.0
        running_loss_D = 0.0
        running_loss_D_real = 0.0
        running_loss_D_fake = 0.0
        running_D_x = 0.0
        running_D_G_z = 0.0
        d_steps = 0
        
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/100')
        for i, (input_imgs, target_imgs) in enumerate(progress_bar):
            input_imgs, target_imgs = input_imgs.to(device), target_imgs.to(device)
            
            # Train Generator
            optimizer_G.zero_grad()
            fake_B = netG(input_imgs)
            fake_AB = torch.cat((input_imgs, fake_B), 1)
            
            pred_fake = netD(fake_AB)
            loss_G_GAN = criterion_GAN(pred_fake, torch.ones_like(pred_fake))
            loss_G_L1 = criterion_L1(fake_B, target_imgs) * lambda_L1
            loss_G = loss_G_GAN + loss_G_L1
            loss_G.backward()
            optimizer_G.step()
            
            running_loss_G += loss_G.item()
            running_D_G_z += torch.sigmoid(pred_fake).mean().item()
            
            # Train Discriminator (every 2nd step)
            if i % 2 == 0:
                optimizer_D.zero_grad()
                
                # Real images
                real_AB = torch.cat((input_imgs, target_imgs), 1)
                pred_real = netD(real_AB)
                real_labels = torch.rand_like(pred_real)*0.1 + 0.9
                loss_D_real = criterion_GAN(pred_real, real_labels)
                
                # Fake images
                fake_AB = torch.cat((input_imgs, fake_B.detach()), 1)
                pred_fake = netD(fake_AB)
                fake_labels = torch.rand_like(pred_fake)*0.1
                loss_D_fake = criterion_GAN(pred_fake, fake_labels)
                
                loss_D = (loss_D_real + loss_D_fake) * 0.5
                loss_D.backward()
                optimizer_D.step()
                
                running_loss_D += loss_D.item()
                running_loss_D_real += loss_D_real.item()
                running_loss_D_fake += loss_D_fake.item()
                running_D_x += torch.sigmoid(pred_real).mean().item()
                d_steps += 1
            
            # Update progress bar
            progress_bar.set_postfix({
                'G': f'{running_loss_G/(i+1):.4f}',
                'D': f'{running_loss_D/d_steps:.4f}' if d_steps > 0 else 'skipped',
                'D(x)': f'{running_D_x/d_steps:.3f}' if d_steps > 0 else '-',
                'D(G(z))': f'{running_D_G_z/(i+1):.3f}'
            })
            
        # Update learning rates
        lr_scheduler_G.step()
        lr_scheduler_D.step()
        
        # Validation
        netG.eval()
        val_loss = 0
        with torch.no_grad():
            for input_imgs, target_imgs in test_loader:
                input_imgs, target_imgs = input_imgs.to(device), target_imgs.to(device)
                fake_B = netG(input_imgs)
                val_loss += criterion_L1(fake_B, target_imgs).item()
            
            # Generate visualization
            fixed_outputs = netG(fixed_inputs)
            visualize_results(fixed_inputs[:3], fixed_targets[:3].to(device), fixed_outputs[:3], epoch)
        
        # Calculate averages
        avg_loss_G = running_loss_G / len(train_loader)
        avg_loss_D = running_loss_D / d_steps if d_steps > 0 else 0
        avg_loss_D_real = running_loss_D_real / d_steps if d_steps > 0 else 0
        avg_loss_D_fake = running_loss_D_fake / d_steps if d_steps > 0 else 0
        avg_D_x = running_D_x / d_steps if d_steps > 0 else 0
        avg_D_G_z = running_D_G_z / len(train_loader)
        avg_val_loss = val_loss / len(test_loader)
        
        # Store history
        history['epoch'].append(epoch+1)
        history['G_loss'].append(avg_loss_G)
        history['D_loss'].append(avg_loss_D)
        history['D_real'].append(avg_loss_D_real)
        history['D_fake'].append(avg_loss_D_fake)
        history['D_x'].append(avg_D_x)
        history['D_G_z'].append(avg_D_G_z)
        history['val_loss'].append(avg_val_loss)
        
        # Print epoch summary
        print(f'\nEpoch {epoch+1} Summary:')
        print(f'G_loss: {avg_loss_G:.4f} | D_loss: {avg_loss_D:.4f}')
        print(f'D_real: {avg_loss_D_real:.4f} | D_fake: {avg_loss_D_fake:.4f}')
        print(f'D(x): {avg_D_x:.3f} | D(G(z)): {avg_D_G_z:.3f}')
        print(f'Val Loss: {avg_val_loss:.4f}')
        
        # Save models
        if (epoch + 1) % 10 == 0:
            torch.save(netG.state_dict(), f'generator_epoch_{epoch+1}.pth')
            torch.save(netD.state_dict(), f'discriminator_epoch_{epoch+1}.pth')
    
    # Save final models
    torch.save(netG.state_dict(), 'generator_final.pth')
    torch.save(netD.state_dict(), 'discriminator_final.pth')
    
    # Plot training history
    plot_training_history(history)

def plot_training_history(history):
    plt.figure(figsize=(15, 10))
    
    # Loss plot
    plt.subplot(2, 2, 1)
    plt.plot(history['epoch'], history['G_loss'], label='Generator Loss')
    plt.plot(history['epoch'], history['D_loss'], label='Discriminator Loss')
    plt.plot(history['epoch'], history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Losses')
    plt.legend()
    
    # D(x) and D(G(z)) plot
    plt.subplot(2, 2, 2)
    plt.plot(history['epoch'], history['D_x'], label='D(x) - Real')
    plt.plot(history['epoch'], history['D_G_z'], label='D(G(z)) - Fake')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.title('Discriminator Output Scores')
    plt.legend()
    
    # D components plot
    plt.subplot(2, 2, 3)
    plt.plot(history['epoch'], history['D_real'], label='D Real Loss')
    plt.plot(history['epoch'], history['D_fake'], label='D Fake Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Discriminator Component Losses')
    plt.legend()
    
    # Save plots
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

if __name__ == "__main__":
    train()

Epoch 1/100: 100%|██████████| 264/264 [01:33<00:00,  2.83it/s, G=7.3922, D=0.6139, D(x)=0.563, D(G(z))=0.432]



Epoch 1 Summary:
G_loss: 7.3922 | D_loss: 0.6139
D_real: 0.6194 | D_fake: 0.6084
D(x): 0.563 | D(G(z)): 0.432
Val Loss: 0.3928


Epoch 2/100: 100%|██████████| 264/264 [01:19<00:00,  3.34it/s, G=8.0334, D=0.4943, D(x)=0.666, D(G(z))=0.336]



Epoch 2 Summary:
G_loss: 8.0334 | D_loss: 0.4943
D_real: 0.5096 | D_fake: 0.4791
D(x): 0.666 | D(G(z)): 0.336
Val Loss: 0.3720


Epoch 3/100: 100%|██████████| 264/264 [01:19<00:00,  3.33it/s, G=8.1279, D=0.5385, D(x)=0.645, D(G(z))=0.353]


...


Epoch 98 Summary:
G_loss: 6.2421 | D_loss: 0.4583
D_real: 0.4575 | D_fake: 0.4592
D(x): 0.670 | D(G(z)): 0.339
Val Loss: 0.3346


Epoch 99/100: 100%|██████████| 264/264 [01:18<00:00,  3.35it/s, G=6.2323, D=0.4565, D(x)=0.671, D(G(z))=0.333]



Epoch 99 Summary:
G_loss: 6.2323 | D_loss: 0.4565
D_real: 0.4549 | D_fake: 0.4582
D(x): 0.671 | D(G(z)): 0.333
Val Loss: 0.3282


Epoch 100/100: 100%|██████████| 264/264 [01:19<00:00,  3.33it/s, G=6.2235, D=0.4590, D(x)=0.666, D(G(z))=0.337]



Epoch 100 Summary:
G_loss: 6.2235 | D_loss: 0.4590
D_real: 0.4599 | D_fake: 0.4582
D(x): 0.666 | D(G(z)): 0.337
Val Loss: 0.3323


In [11]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [12]:
len(test_loader)

264

In [17]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision.utils as vutils
import os
from tqdm import tqdm
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

def inference(checkpoint_path, input_dir, target_dir, save_dir='predictions', num_samples=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Загрузка модели
    netG = UNet().to(device)
    netG.load_state_dict(torch.load(checkpoint_path, map_location=device))
    netG.eval()


    os.makedirs(save_dir, exist_ok=True)
    mask_dir = os.path.join(save_dir, 'predicted_masks')
    os.makedirs(mask_dir, exist_ok=True)

    with torch.no_grad():
        for idx, (input_img, target_img) in tqdm(enumerate(test_loader), total=len(test_loader)):
            input_img = input_img.to(device)
            fake_mask = netG(input_img)

            # Сопоставим имя файла с оригинальным индексом
            real_idx = train_size + idx
            base_name = os.path.splitext(dataset.filenames[real_idx])[0]  # например: "img_012"

            # Обработка изображений
            fake_mask_img = fake_mask.squeeze(0).squeeze(0).cpu()       # [1, 1, H, W] → [H, W]
            input_vis = input_img.squeeze(0).cpu()                      # [1, C, H, W] → [C, H, W]
            target_vis = target_img.squeeze(0).squeeze(0).cpu()         # [1, 1, H, W] → [H, W]

            # Сохраняем отдельную предсказанную маску
            plt.imsave(os.path.join(mask_dir, f'{base_name}_pred.png'), fake_mask_img, cmap='gray')

            # Визуализация
            fig, axs = plt.subplots(1, 3, figsize=(12, 4))

            if input_vis.shape[0] == 3:  # RGB
                axs[0].imshow(input_vis.permute(1, 2, 0) * 0.5 + 0.5)
            else:  # Grayscale
                axs[0].imshow(input_vis.squeeze(), cmap='gray')
            axs[0].set_title('Input')

            axs[1].imshow(target_vis, cmap='gray')
            axs[1].set_title('Ground Truth')

            axs[2].imshow(fake_mask_img, cmap='gray')
            axs[2].set_title('Predicted Mask')

            for ax in axs:
                ax.axis('off')

            plt.tight_layout()
            plt.savefig(f'{save_dir}/sample_{idx:03d}.png')
            plt.close()

            if idx + 1 >= num_samples:
                break


if __name__ == '__main__':
    inference(
        checkpoint_path='generator_final.pth',
        input_dir='/kaggle/input/dataset1/sliced images/input',
        target_dir='/kaggle/input/dataset1/sliced images/target',
        save_dir='predictions',
        num_samples=264
    )

/tmp/ipykernel_31/2036143785.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  netG.load_state_dict(torch.load(checkpoint_path, map_location=device))
100%|█████████▉| 263

In [23]:
import torch
from torch.utils.data import DataLoader, random_split
import os
from tqdm import tqdm
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim_func
from math import log10
import csv

def compute_metrics(pred, target):
    pred = (pred > 0).astype(np.uint8)
    target = (target > 0).astype(np.uint8)

    intersection = np.logical_and(pred, target).sum()
    union = np.logical_or(pred, target).sum()

    dice = 2 * intersection / (pred.sum() + target.sum() + 1e-8)
    iou = intersection / (union + 1e-8)
    mae = np.mean(np.abs(pred - target))
    rmse = np.sqrt(np.mean((pred - target) ** 2))
    ssim = ssim_func(target, pred, data_range=1)

    mse = np.mean((pred - target) ** 2)
    psnr = 20 * log10(1.0) - 10 * log10(mse + 1e-8)  # Assuming pixel values in [0, 1]

    return dice, iou, mae, rmse, ssim, psnr

def inference_with_metrics(checkpoint_path, input_dir, target_dir, save_dir='predictions'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Загрузка модели
    netG = UNet().to(device)
    netG.load_state_dict(torch.load(checkpoint_path, map_location=device))
    netG.eval()

    os.makedirs(save_dir, exist_ok=True)

    total_dice = total_iou = total_mae = total_rmse = total_ssim = total_psnr = 0

    with torch.no_grad():
        for idx, (input_img, target_img) in tqdm(enumerate(test_loader), total=len(test_loader)):
            input_img = input_img.to(device)
            fake_mask = netG(input_img)

            fake_mask_np = fake_mask.squeeze().cpu().numpy()
            target_np = target_img.squeeze().cpu().numpy()

            # Бинаризация
            fake_mask_np_bin = (fake_mask_np > 0).astype(np.uint8)
            target_np_bin = (target_np > 0).astype(np.uint8)

            # Метрики
            dice, iou, mae, rmse, ssim, psnr = compute_metrics(fake_mask_np_bin, target_np_bin)
            total_dice += dice
            total_iou += iou
            total_mae += mae
            total_rmse += rmse
            total_ssim += ssim
            total_psnr += psnr

    N = len(test_loader)
    avg_dice = total_dice / N
    avg_iou = total_iou / N
    avg_mae = total_mae / N
    avg_rmse = total_rmse / N
    avg_ssim = total_ssim / N
    avg_psnr = total_psnr / N

    print(f"\n📊 Metrics on Test Set:")
    print(f"Dice:  {avg_dice:.4f}")
    print(f"IoU:   {avg_iou:.4f}")
    print(f"MAE:   {avg_mae:.4f}")
    print(f"RMSE:  {avg_rmse:.4f}")
    print(f"SSIM:  {avg_ssim:.4f}")
    print(f"PSNR:  {avg_psnr:.4f}")

    # Сохранение в CSV
    csv_path = os.path.join(save_dir, 'metrics.csv')
    with open(csv_path, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Dice', avg_dice])
        writer.writerow(['IoU', avg_iou])
        writer.writerow(['MAE', avg_mae])
        writer.writerow(['RMSE', avg_rmse])
        writer.writerow(['SSIM', avg_ssim])
        writer.writerow(['PSNR', avg_psnr])

if __name__ == '__main__':
    inference_with_metrics(
        checkpoint_path='generator_final.pth',
        input_dir='/kaggle/input/dataset1/sliced images/input',
        target_dir='/kaggle/input/dataset1/sliced images/target',
        save_dir='predictions'
    )

/tmp/ipykernel_31/4276485695.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  netG.load_state_dict(torch.load(checkpoint_path, map_location=device))
100%|██████████| 264


📊 Metrics on Test Set:
Dice:  0.8333
IoU:   0.7273
MAE:   21.0411
RMSE:  0.3929
SSIM:  0.5130
PSNR:  8.3518


In [24]:
import shutil

shutil.make_archive('/kaggle/working/output', 'zip', '/kaggle/working')

'/kaggle/working/output.zip'